In [ ]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

import json

from time import sleep

import web_crawler.scraper_walmart as walmart

# Our generated code
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2

In [ ]:
products = pd.read_pickle("./data/products_pd.pickle")

products['price_dollar'] = np.nan



View the products from Walmart stores

In [ ]:
products.loc[products["stores"].apply({"Migros"}.issubset  {"Monoprix"}.issubset |
                                      {"Coop"}.issubset |
                                      {"Cora"}.issubset)]

Request products details from Walmart Open API

In [ ]:
products.product_name = products.product_name.fillna("")
products.brands = products.brands.fillna("")

scraped_products = []

with open('walmart_products.json', mode='w', encoding='utf-8') as f:
    json.dump([], f)

    count = 0
    for i, row in products.iterrows():
        if "Walmart" in row.stores:
            count = count+1
            query = row['product_name'] + row["brands"]
            print(query.title())
            try:
                products_dicts = walmart.search(query)
            except Exception:
                continue

            for product_dict in products_dicts:
                item = {'code': row.name, 
                        'product_name': row.product_name,
                        'store_product_name': product_dict.get('name', ""),
                        'store_price': product_dict.get('salePrice', ""),
                        'store_currency': 'USD',
                        'store_category_path' : product_dict.get('categoryPath', "")
                        }
                scraped_products.append(item)
                
            # Cache intermediate results
            if not count%100:
                json.dump(scraped_products, f)
            
            # We don't wanna get banned from the server
            sleep(0.1)

    json.dump(scraped_products, f)

In [ ]:
with open('walmart_products.json', mode='w', encoding='utf-8') as f:
    json.dump(scraped_products, f)

In [ ]:
count

In [ ]:
products_dicts[0].keys()

In [ ]:
products_dicts[0]['categoryPath']